In [1]:
from sklearn.datasets import load_files
from keras.utils import np_utils
from keras.utils import to_categorical
import numpy as np
from glob import glob

def load_dataset(path):
    data=load_files(path)#读取了整个文件夹 所以data是类似字典的数据类型
    dog_files=np.array(data['filenames'])#返回所有目录下照片的文件名
    dog_targets=to_categorical(np.array(data['target']),133)#返回0-132类别编号
    return dog_files,dog_targets

Using TensorFlow backend.


In [2]:
train_files,train_targets=load_dataset(r'C:\\Users\\wlwy\\Documents\\dogImages\\dogImages\\train')
valid_files,valid_targets=load_dataset(r'C:\\Users\\wlwy\\Documents\\dogImages\\dogImages\valid')
test_files,test_targets  =load_dataset(r'C:\\Users\wlwy\\Documents\\dogImages\\dogImages\\test')

dog_names=[item[25:-1] for item in glob('C:\\Users\\wlwy\\Documents\\dogImages\\dogImages\\train\\*')]

In [3]:
#读取之前经过训练好的数据，穿过了前面的Vgg16的多层  相当于冻结前面权重
bottleneck_features=np.load('C:\\Users\\wlwy\\Documents\\DogVGG16Data.npz')
train_vgg16=bottleneck_features['train']
valid_vgg16=bottleneck_features['valid']
test_vgg16 =bottleneck_features['test']

In [4]:
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D

model = Sequential()
model.add(GlobalAveragePooling2D(input_shape=(7, 7, 512)))#增加了全局优化池GAP，用来减少参数，防止过拟合，即每7*7个方格中取一个
model.add(Dense(133, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 133)               68229     
Total params: 68,229
Trainable params: 68,229
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', 
                  metrics=['accuracy'])
from keras.callbacks import ModelCheckpoint   

# train the model
checkpointer = ModelCheckpoint(filepath='dogvgg16.weights.best.hdf5', verbose=1, 
                               save_best_only=True)
model.fit(train_vgg16, train_targets, epochs=25, validation_data=(valid_vgg16, valid_targets), 
          callbacks=[checkpointer], verbose=1, shuffle=True)

Train on 6680 samples, validate on 835 samples
Epoch 1/25
6680/6680 [==============================] - 2s 233us/step - loss: 6.0034 - acc: 0.6102 - val_loss: 6.9562 - val_acc: 0.4743

Epoch 00001: val_loss improved from inf to 6.95624, saving model to dogvgg16.weights.best.hdf5
Epoch 2/25
6680/6680 [==============================] - 1s 212us/step - loss: 5.8675 - acc: 0.6208 - val_loss: 6.8331 - val_acc: 0.4862

Epoch 00002: val_loss improved from 6.95624 to 6.83307, saving model to dogvgg16.weights.best.hdf5
Epoch 3/25
6680/6680 [==============================] - 1s 221us/step - loss: 5.6681 - acc: 0.6314 - val_loss: 6.7135 - val_acc: 0.4910

Epoch 00003: val_loss improved from 6.83307 to 6.71353, saving model to dogvgg16.weights.best.hdf5
Epoch 4/25
6680/6680 [==============================] - 1s 218us/step - loss: 5.6011 - acc: 0.6406 - val_loss: 6.6686 - val_acc: 0.4850

Epoch 00004: val_loss improved from 6.71353 to 6.66863, saving model to dogvgg16.weights.best.hdf5
Epoch 5/25
66

In [8]:
model.load_weights('dogvgg16.weights.best.hdf5')
vgg16_predictions = [np.argmax(model.predict(np.expand_dims(feature, axis=0))) #argmax为取最大元素的索引值，即返回热独编码前的0-133标签编号
                     for feature in test_vgg16]
#vgg16_predictions得到每组数据对应的类别编号
# report test accuracy
test_accuracy = 100*np.sum(np.array(vgg16_predictions)==
                           np.argmax(test_targets, axis=1))/len(vgg16_predictions)
print('\nTest accuracy: %.4f%%' % test_accuracy)
#精度提升  确实存在过拟合现象


Test accuracy: 54.7847%


In [9]:
vgg16_predictions

[56,
 69,
 114,
 18,
 54,
 92,
 44,
 51,
 124,
 50,
 1,
 18,
 93,
 132,
 70,
 44,
 83,
 68,
 114,
 41,
 3,
 68,
 60,
 95,
 9,
 130,
 18,
 18,
 21,
 112,
 4,
 108,
 28,
 11,
 105,
 44,
 72,
 38,
 3,
 117,
 69,
 84,
 41,
 26,
 76,
 1,
 81,
 89,
 87,
 7,
 73,
 119,
 73,
 115,
 87,
 99,
 69,
 103,
 29,
 2,
 10,
 102,
 33,
 95,
 81,
 27,
 132,
 95,
 115,
 37,
 8,
 57,
 81,
 69,
 108,
 76,
 29,
 130,
 37,
 95,
 115,
 102,
 68,
 93,
 92,
 106,
 11,
 45,
 5,
 84,
 93,
 14,
 130,
 101,
 21,
 21,
 95,
 101,
 46,
 83,
 7,
 10,
 55,
 81,
 71,
 114,
 51,
 56,
 83,
 86,
 98,
 62,
 49,
 11,
 3,
 10,
 57,
 82,
 27,
 43,
 117,
 0,
 131,
 98,
 82,
 109,
 89,
 78,
 43,
 28,
 47,
 117,
 26,
 109,
 69,
 1,
 70,
 55,
 117,
 49,
 47,
 74,
 47,
 55,
 4,
 2,
 83,
 43,
 87,
 6,
 87,
 57,
 6,
 82,
 108,
 1,
 95,
 9,
 85,
 112,
 49,
 28,
 114,
 87,
 95,
 37,
 72,
 21,
 115,
 116,
 72,
 120,
 44,
 59,
 1,
 62,
 119,
 1,
 111,
 87,
 76,
 99,
 6,
 6,
 49,
 121,
 45,
 6,
 4,
 47,
 6,
 47,
 105,
 123,
 1,
 45,
 60,
 1